# Sufficient levels computation variants
We have two variants how to compute the $\mathrm{Safe}^\mathit{cap}$ vector (level of energy needed to survive with given capacity).

1. Iterate minInitCons (largest fixpoint) on MDP where reloads with ∞ are removed. \[$n^2$ iterations\]
2. Least fixpoint from minInitCons \[$(n\cdot c_{\mathrm{max}})$ iterations\]


The two variants are implemented currently in energy_solver.py in:
1. 
```python
class BasicES
```

2. 
```python
class LeastFixpointES
```

The option 2 computes minInitCons as a first step anyway (no reloads are removed at the beginning) as it results in a cleaner code. However, it does not sotre the values.

In [1]:
from fimdp.energy_solvers import BasicES, LeastFixpointES
from fimdp.objectives import *

## Examples
* Manhattan, NYC AEV Example.
* a tiny MDP used to show incorrectness of the least-fixpoint with at most $|S|$ iterations.

### Manhattan example

### Tiny example

In [2]:
from fimdp.core import ConsMDP
m = ConsMDP()
m.new_state(True)
m.new_states(2)
m.new_state(True)
m.add_action(0, {0:1}, "", 1)
m.add_action(1, {0:1}, "a", 1000)
m.add_action(1, {2:1}, "b", 1)
m.add_action(2, {1:1}, "b", 1)
m.add_action(3, {3:1}, "r", 1010)
m.add_action(1, {3:1}, "r", 1)
m.add_action(2, {3:1}, "r", 1);

## Variant 1

In [3]:
basic = BasicES(m, 1005, None)
%timeit basic.get_min_levels(SAFE, recompute=True)

108 µs ± 1.15 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Variant 2

In [4]:
least = LeastFixpointES(m, 1005, None)
%timeit least.get_min_levels(SAFE, recompute=True)

9.02 ms ± 817 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Effect on Büchi computation is negligible as it does not use get_min_levels(SAFE)